Pantaree Somcharoen (G00365378)

# Countdown Numbers Game Notebook
***

## What is a Countdown Game?
***

<b>Countdown</b> is a game show of letters, words & numbers. It is based on the long-running French game show Des chiffres et des lettres ("Numbers & Letters"). [[1]](https://british-game-show.fandom.com/wiki/Countdown)

<img src="https://149458054.v2.pressablecdn.com/wp-content/uploads/2017/12/countdown-game-printable-free.png" width=500 height=500 />

### Countdown Numbers game 
***



The Countdown Numbers problem is as follows: given a sequence of source numbers and the target number, the basic arithmetical operators (addition, subtraction,
multiplication and division) solves the problem.[[2]](https://www.cambridge.org/core/journals/journal-of-functional-programming/article/countdown-problem/FCF81E290B194D1ACBF699842D4D4CA5)

There are always six source numbers selected from the sequence[1..10,1..10,25,50,75,100], the target number is randomly chosen from the range 100..999 approximate solutions are acceptable, each number can only be used once, and
there is a time limit of 30 seconds.[[3]](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.86.1168&rep=rep1&type=pdf)

<img src="https://i.ytimg.com/vi/sCqNY-k9jho/maxresdefault.jpg" width=500 height=500 />

For example[[4]](https://arxiv.org/pdf/1502.05450.pdf), if the six numbers drawn are
1,1,4,5,6,7 and the number to find is 899 the answer is: <br>
<b>Operations Remaining</b><br>
6 x 5 = 30 {1,1,4,7,30} <br>
30 + 1 = 31 {1,4,7,31}<br>
4 x 7 = 28 {1,28,31}<br>
28 + 1 = 29 {29,31}<br>
29 * 31 = 899 {899}<br>

## Code


#### Setup
***

In [113]:
# Permutations and combinations.
import itertools as it

In [114]:
# Random number generation.
import random

In [115]:
# Operators as functions.
import operator

#### Simulate a game
***

In [116]:
# The large numbers.
large = [25, 50, 75, 100]
large

[25, 50, 75, 100]

In [117]:
# The small numbers.
small = list(range(1, 11)) * 2
small

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [118]:
# The number of large numbers to pick - between 0 and 4 inclusive.
no_large = random.randrange(0, 5)
no_large

1

In [119]:
# Select no_large large numbers at random.
large_rand = random.sample(large, no_large)
large_rand

[75]

In [120]:
# Select (6 - no_large) small numbers at random.
small_rand = random.sample(small, 6 - no_large)
small_rand

[3, 5, 2, 4, 7]

In [121]:
# The six random numbers in a list.
play_nos = large_rand + small_rand
play_nos

[75, 3, 5, 2, 4, 7]

In [122]:
# Pick a random target number.
target = random.randrange(101, 1000)
target

559

In [123]:
# All in one function.
# For random nubmers and samples.
import random

def new_numbers_game(no_large=None):
  """ Returns six numbers and a target number representing a Countdown numbers game.
  """
  # If no_large in None, randomly pick value between 0 and 4 inclusive.
  if no_large is None:
    # Randomly set the value.
    no_large = random.randrange(0, 5)
  
  # Select random large numbers.
  large_rand = random.sample([25, 50, 75, 100], no_large)
  # Select random small numbers.
  small_rand = random.sample(list(range(1, 11)) * 2, 6 - no_large)
  # The playing numbers.
  play_nos = large_rand + small_rand

  # Select a target number.
  target = random.randrange(101, 1000)

  # Return the game.
  return play_nos, target

In [124]:
# Random numb ers game.
new_numbers_game()

([100, 25, 9, 2, 10, 3], 235)

#### Working Towards a Solution
***

In [125]:
# A new example game.
play_nos, target = new_numbers_game()
play_nos, target

([75, 100, 50, 9, 3, 6], 927)

In [126]:
# Looping through all permutations of two playing numbers.
for p in it.permutations(play_nos, 2):
  # Print the two numbers.
  print(p)
  # Print their sum.
  print(f'{p[0]} + {p[1]} = {p[0]+p[1]}')
  # Print their product.
  print(f'{p[0]} * {p[1]} = {p[0]*p[1]}')
  # Print their difference if it is positive.
  if p[0] - p[1] > 0:
    print(f'{p[0]} - {p[1]} = {p[0]-p[1]}')
  # Print their quotient if it is an integer.
  if p[0] % p[1] == 0:
    print(f'{p[0]} / {p[1]} = {p[0]//p[1]}')
  # Print a blank line.
  print()

(75, 100)
75 + 100 = 175
75 * 100 = 7500

(75, 50)
75 + 50 = 125
75 * 50 = 3750
75 - 50 = 25

(75, 9)
75 + 9 = 84
75 * 9 = 675
75 - 9 = 66

(75, 3)
75 + 3 = 78
75 * 3 = 225
75 - 3 = 72
75 / 3 = 25

(75, 6)
75 + 6 = 81
75 * 6 = 450
75 - 6 = 69

(100, 75)
100 + 75 = 175
100 * 75 = 7500
100 - 75 = 25

(100, 50)
100 + 50 = 150
100 * 50 = 5000
100 - 50 = 50
100 / 50 = 2

(100, 9)
100 + 9 = 109
100 * 9 = 900
100 - 9 = 91

(100, 3)
100 + 3 = 103
100 * 3 = 300
100 - 3 = 97

(100, 6)
100 + 6 = 106
100 * 6 = 600
100 - 6 = 94

(50, 75)
50 + 75 = 125
50 * 75 = 3750

(50, 100)
50 + 100 = 150
50 * 100 = 5000

(50, 9)
50 + 9 = 59
50 * 9 = 450
50 - 9 = 41

(50, 3)
50 + 3 = 53
50 * 3 = 150
50 - 3 = 47

(50, 6)
50 + 6 = 56
50 * 6 = 300
50 - 6 = 44

(9, 75)
9 + 75 = 84
9 * 75 = 675

(9, 100)
9 + 100 = 109
9 * 100 = 900

(9, 50)
9 + 50 = 59
9 * 50 = 450

(9, 3)
9 + 3 = 12
9 * 3 = 27
9 - 3 = 6
9 / 3 = 3

(9, 6)
9 + 6 = 15
9 * 6 = 54
9 - 6 = 3

(3, 75)
3 + 75 = 78
3 * 75 = 225

(3, 100)
3 + 100 = 103
3 * 10

#### Operators and Functions
***

In [127]:
# The + operator as a function.
operator.add(4, 5)

9

In [128]:
# The * operator as a function.
operator.mul(4, 5)

20

In [129]:
# The - operator as a function.
operator.sub(4, 5)

-1

In [130]:
# The / operator as a function.
operator.truediv(4, 5)

0.8

In [131]:
# The benefit of these is that they are first class objects.
# Note the +, -, *, / operators can't be put in lists.
ops = [operator.add, operator.mul, operator.sub, operator.truediv] * 5
ops

[<function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>]

In [132]:
# Using permutations we can get all permutations 
# with replacement of five operations.

# We use a limit because their are a large number.
limit = 10


for q in it.permutations(ops * 5, 5):
  if limit == 0:
    break
  print(q)
  limit = limit - 1

(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function add>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function mul>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function sub>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function truediv>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function add>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function mul>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function sub>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in 

#### Permutations and Combinations
***

In [133]:
# Example of combinations.
# Order matters: no. Replacement: no.
L = [1, 2, 3, 4]
for c in it.combinations(L, 2):
  print(c)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [134]:
# Example of combinations with replacement.
# Order matters: no. Replacement: yes.
L = [1, 2, 3, 4]
for c in it.combinations_with_replacement(L, 2):
  print(c)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 2)
(2, 3)
(2, 4)
(3, 3)
(3, 4)
(4, 4)


In [135]:
# Example of permutations of size 2.
# Order matters: yes. Replacement: no.
L = [1, 2, 3, 4]
for c in it.permutations(L, 2):
  print(c)

(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 4)
(4, 1)
(4, 2)
(4, 3)


In [136]:
# Example of products of length 2.
# Order matters: yes. Replacement: yes.
L = [1, 2, 3, 4]
for c in it.product(L, repeat=2):
  print(c)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(4, 1)
(4, 2)
(4, 3)
(4, 4)


In [137]:
# Using product to generate all lists of five operations.
ops = [operator.add, operator.mul, operator.sub, operator.truediv]
limit = 100
for q in it.product(ops, repeat=5):
  if limit == 0:
    break
  print(q)
  limit = limit - 1

(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function truediv>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function add>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function mul>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function sub>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in fun

#### Reverse Polish Notation
***

In [138]:
# 3 4 5 * +
3 + 4 * 5

23

In [139]:
# 3 4 + 5 *
(3 + 4) * 5

35

In [140]:
# New random numbers game.
play_nos, target = new_numbers_game()
play_nos, target

([75, 50, 100, 2, 8, 5], 178)

In [141]:
# Orderings of pairs.
for pair in it.permutations(play_nos, 2):
  print(pair)

(75, 50)
(75, 100)
(75, 2)
(75, 8)
(75, 5)
(50, 75)
(50, 100)
(50, 2)
(50, 8)
(50, 5)
(100, 75)
(100, 50)
(100, 2)
(100, 8)
(100, 5)
(2, 75)
(2, 50)
(2, 100)
(2, 8)
(2, 5)
(8, 75)
(8, 50)
(8, 100)
(8, 2)
(8, 5)
(5, 75)
(5, 50)
(5, 100)
(5, 2)
(5, 8)


In [142]:
# Change the target to be something that will work for just two numbers.
target = max(play_nos) * min(play_nos)
target

200

In [143]:
%%timeit

# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# All pair, op combs that hit target.
for nos, op in it.product(it.permutations(play_nos, 2), ops):
  if op(nos[0], nos[1]) == target:
    print(nos[0], str(op), nos[1])

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

...

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

100 <built-in function mul> 2
2 <built-in function mul> 100
713 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [144]:
%%timeit

# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

def hits_target(z):
  nos, op = z
  return (op(nos[0], nos[1]) == target)

# All pair, op combs that hit target.
list(filter(hits_target, it.product(it.permutations(play_nos, 2), ops)))

28.2 µs ± 467 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [145]:
# All pair, op combs that hit target.
list(filter(lambda z: z[1](z[0][0], z[0][1]) == target, it.product(it.permutations(play_nos, 2), ops)))

[((100, 2), <function _operator.mul(a, b, /)>),
 ((2, 100), <function _operator.mul(a, b, /)>)]

In [146]:
# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# Limit the output.
limit = 1100

# For the limit.
i = 0
# Orderings of pairs.
for play_nos, opers in it.product(it.permutations(play_nos), it.product(*([ops] * 5))):
  print(play_nos, opers)
  i = i + 1
  if i >= limit:
    break

(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <bu

(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function sub>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function sub>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function sub>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function mul>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function mul>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function mul>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <

(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function sub>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function add>)
(75, 50, 100, 2, 8, 5)

In [147]:
# Number of combinations of 5 operators with replacement.
4**5

1024

In [148]:
# No of permutations of playing numbers.
import math
math.factorial(6)

720

In [149]:
4**5 * math.factorial(6)

737280

In [150]:
# We (might not have/) haven't considered all combinations:
# RPN with (1, 2, 3, 4) and (+, -, +)...
# 1 2 3 4 + - + (-4)
# 1 2 + 3 4 - + (2)
# 1 2 3 + - 4 + (0)

#### Partitions
***

In [151]:
# An example list of six numbers.
numbers = [100, 75, 10, 4, 2, 1]

In [152]:
# Give all 2-partitions of a list
# where each sublist has at least one element.
def partitions(L):
  for i in range(1, len(L)):
    # Slice the list using i.
    print(L[:i], L[i:])

In [153]:
# All partitions of the numbers list.
partitions(numbers)

[100] [75, 10, 4, 2, 1]
[100, 75] [10, 4, 2, 1]
[100, 75, 10] [4, 2, 1]
[100, 75, 10, 4] [2, 1]
[100, 75, 10, 4, 2] [1]


In [154]:
# Some of the sublists in turn can be partitioned.
partitions(numbers[1:])

[75] [10, 4, 2, 1]
[75, 10] [4, 2, 1]
[75, 10, 4] [2, 1]
[75, 10, 4, 2] [1]


In [155]:
# And some of the sublists can be further (and further) parititioned>
partitions(numbers[2:])

[10] [4, 2, 1]
[10, 4] [2, 1]
[10, 4, 2] [1]


In [156]:
# We'll use generators in this circumstance.
range(100000000000)

range(0, 100000000000)

In [157]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def partitions(L):
  # Check if there is no way to partition further.
  if len(L) == 1:
    yield f"{L[0]}"
  for i in range(1, len(L)):
    # Slice the list using i.
    for left, right in it.product(partitions(L[:i]), partitions(L[i:])):
      yield f"({left} ? {right})"

In [158]:
total = 0
for i in partitions(numbers):
  print(i)
  total = total + 1
print(total)

(100 ? (75 ? (10 ? (4 ? (2 ? 1)))))
(100 ? (75 ? (10 ? ((4 ? 2) ? 1))))
(100 ? (75 ? ((10 ? 4) ? (2 ? 1))))
(100 ? (75 ? ((10 ? (4 ? 2)) ? 1)))
(100 ? (75 ? (((10 ? 4) ? 2) ? 1)))
(100 ? ((75 ? 10) ? (4 ? (2 ? 1))))
(100 ? ((75 ? 10) ? ((4 ? 2) ? 1)))
(100 ? ((75 ? (10 ? 4)) ? (2 ? 1)))
(100 ? (((75 ? 10) ? 4) ? (2 ? 1)))
(100 ? ((75 ? (10 ? (4 ? 2))) ? 1))
(100 ? ((75 ? ((10 ? 4) ? 2)) ? 1))
(100 ? (((75 ? 10) ? (4 ? 2)) ? 1))
(100 ? (((75 ? (10 ? 4)) ? 2) ? 1))
(100 ? ((((75 ? 10) ? 4) ? 2) ? 1))
((100 ? 75) ? (10 ? (4 ? (2 ? 1))))
((100 ? 75) ? (10 ? ((4 ? 2) ? 1)))
((100 ? 75) ? ((10 ? 4) ? (2 ? 1)))
((100 ? 75) ? ((10 ? (4 ? 2)) ? 1))
((100 ? 75) ? (((10 ? 4) ? 2) ? 1))
((100 ? (75 ? 10)) ? (4 ? (2 ? 1)))
((100 ? (75 ? 10)) ? ((4 ? 2) ? 1))
(((100 ? 75) ? 10) ? (4 ? (2 ? 1)))
(((100 ? 75) ? 10) ? ((4 ? 2) ? 1))
((100 ? (75 ? (10 ? 4))) ? (2 ? 1))
((100 ? ((75 ? 10) ? 4)) ? (2 ? 1))
(((100 ? 75) ? (10 ? 4)) ? (2 ? 1))
(((100 ? (75 ? 10)) ? 4) ? (2 ? 1))
((((100 ? 75) ? 10) ? 4) ? (

In [159]:
# Example of ((100 ? 75) ? (10 ? (4 ? (2 ? 1)))).
((100 + 75) - (10 * (4 - (2 + 1))))

165

In [160]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def patterns(numbers, operators):
  # Check if there is no way to partition further.
  if len(numbers) == 1:
    yield numbers[0]
  # Loop through all the ways to partition L into two non-empty sublists.
  for i in range(1, len(numbers)):
    # Slice the list using i.
    for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
      # Yield the next operator applied to the sublists.
      yield f'({left} {operators[0]} {right})' #[left, operators[0], right]

In [161]:
# An example list of six numbers.
numbers = [100, 75, 10, 4, 2, 1]
# Example operators.
# operators = [operator.add, operator.mul, operator.sub, operator.add, operator.add]
operators = ['+', '*', '-', '+', '+']

In [162]:
# Using eval, which mightn't be great.
for i in patterns(numbers, operators):
  print(f'{i} = {eval(i)}')

(100 + (75 * (10 - (4 + (2 + 1))))) = 325
(100 + (75 * (10 - ((4 + 2) + 1)))) = 325
(100 + (75 * ((10 + 4) - (2 + 1)))) = 925
(100 + (75 * ((10 + (4 + 2)) - 1))) = 1225
(100 + (75 * (((10 + 4) + 2) - 1))) = 1225
(100 + ((75 - 10) * (4 + (2 + 1)))) = 555
(100 + ((75 - 10) * ((4 + 2) + 1))) = 555
(100 + ((75 - (10 + 4)) * (2 + 1))) = 283
(100 + (((75 + 10) - 4) * (2 + 1))) = 343
(100 + ((75 - (10 + (4 + 2))) * 1)) = 159
(100 + ((75 - ((10 + 4) + 2)) * 1)) = 159
(100 + (((75 + 10) - (4 + 2)) * 1)) = 179
(100 + (((75 + (10 + 4)) - 2) * 1)) = 187
(100 + ((((75 + 10) + 4) - 2) * 1)) = 187
((100 * 75) + (10 - (4 + (2 + 1)))) = 7503
((100 * 75) + (10 - ((4 + 2) + 1))) = 7503
((100 * 75) + ((10 + 4) - (2 + 1))) = 7511
((100 * 75) + ((10 + (4 + 2)) - 1)) = 7515
((100 * 75) + (((10 + 4) + 2) - 1)) = 7515
((100 * (75 - 10)) + (4 + (2 + 1))) = 6507
((100 * (75 - 10)) + ((4 + 2) + 1)) = 6507
(((100 - 75) * 10) + (4 + (2 + 1))) = 257
(((100 - 75) * 10) + ((4 + 2) + 1)) = 257
((100 * (75 - (10 + 4))) 

#### RPN and Patterns
***

In [163]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def patterns(numbers, operators):
  # Check if there is no way to partition further.
  if len(numbers) == 1:
    yield numbers
  # Loop through all the ways to partition L into two non-empty sublists.
  for i in range(1, len(numbers)):
    # Slice the list using i.
    for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
      # Yield the next operator applied to the sublists.
      yield [*left, *right, operators[0]]

In [164]:
# An example list of six numbers.
numbers = [100, 75, 10, 4, 2, 1]
# Example operators.
# operators = [operator.add, operator.mul, operator.sub, operator.add, operator.add]
operators = ['+', '*', '-', '+', '+']
# Using eval, which mightn't be great.
for i in patterns(numbers, operators):
  print(i)

[100, 75, 10, 4, 2, 1, '+', '+', '-', '*', '+']
[100, 75, 10, 4, 2, '+', 1, '+', '-', '*', '+']
[100, 75, 10, 4, '+', 2, 1, '+', '-', '*', '+']
[100, 75, 10, 4, 2, '+', '+', 1, '-', '*', '+']
[100, 75, 10, 4, '+', 2, '+', 1, '-', '*', '+']
[100, 75, 10, '-', 4, 2, 1, '+', '+', '*', '+']
[100, 75, 10, '-', 4, 2, '+', 1, '+', '*', '+']
[100, 75, 10, 4, '+', '-', 2, 1, '+', '*', '+']
[100, 75, 10, '+', 4, '-', 2, 1, '+', '*', '+']
[100, 75, 10, 4, 2, '+', '+', '-', 1, '*', '+']
[100, 75, 10, 4, '+', 2, '+', '-', 1, '*', '+']
[100, 75, 10, '+', 4, 2, '+', '-', 1, '*', '+']
[100, 75, 10, 4, '+', '+', 2, '-', 1, '*', '+']
[100, 75, 10, '+', 4, '+', 2, '-', 1, '*', '+']
[100, 75, '*', 10, 4, 2, 1, '+', '+', '-', '+']
[100, 75, '*', 10, 4, 2, '+', 1, '+', '-', '+']
[100, 75, '*', 10, 4, '+', 2, 1, '+', '-', '+']
[100, 75, '*', 10, 4, 2, '+', '+', 1, '-', '+']
[100, 75, '*', 10, 4, '+', 2, '+', 1, '-', '+']
[100, 75, 10, '-', '*', 4, 2, 1, '+', '+', '+']
[100, 75, 10, '-', '*', 4, 2, '+', 1, '+

In [165]:
# An example list of six numbers.
numbers = [100, 75, 10, 4, 2, 1]
# Example operators.
operators = [operator.add, operator.mul, operator.sub, operator.add, operator.add]
# operators = ['+', '*', '-', '+', '+']
# Using eval, which mightn't be great.
for i in patterns(numbers, operators):
  print(i)

[100, 75, 10, 4, 2, 1, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
[100, 75, 10, 4, 2, <built-in function add>, 1, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
[100, 75, 10, 4, <built-in function add>, 2, 1, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
[100, 75, 10, 4, 2, <built-in function add>, <built-in function add>, 1, <built-in function sub>, <built-in function mul>, <built-in function add>]
[100, 75, 10, 4, <built-in function add>, 2, <built-in function add>, 1, <built-in function sub>, <built-in function mul>, <built-in function add>]
[100, 75, 10, <built-in function sub>, 4, 2, 1, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function add>]
[100, 75, 10, <built-in function sub>, 4, 2, <built-in function add>, 1, <built-in function add>, <built-in func

In [166]:
# Evaluate RPN expression.
def eval_rpn(rpn):
  # A stack.
  stack = []
  # Loop through rpn an item at a time.
  for i in rpn:
    # Check if it's a number.
    if isinstance(i, int):
      # Append to the stack.
      stack = stack + [i]
    else:
      # Pop from stack twice.
      right = stack[-1]
      stack = stack[:-1]
      left = stack[-1]
      stack = stack[:-1]
      # Push operator applied to stack elements.
      stack = stack + [i(left, right)]
  # Should only be one item on stack.
  return stack[0]

In [167]:
# An example list of six numbers.
numbers = [100, 75, 10, 4, 2, 1]

# Example operators.
operators = [operator.add, operator.mul, operator.sub, operator.add, operator.add]

# Using eval, which mightn't be great.
for i in patterns(numbers, operators):
  print(eval_rpn(i), i)

325 [100, 75, 10, 4, 2, 1, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
325 [100, 75, 10, 4, 2, <built-in function add>, 1, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
925 [100, 75, 10, 4, <built-in function add>, 2, 1, <built-in function add>, <built-in function sub>, <built-in function mul>, <built-in function add>]
1225 [100, 75, 10, 4, 2, <built-in function add>, <built-in function add>, 1, <built-in function sub>, <built-in function mul>, <built-in function add>]
1225 [100, 75, 10, 4, <built-in function add>, 2, <built-in function add>, 1, <built-in function sub>, <built-in function mul>, <built-in function add>]
555 [100, 75, 10, <built-in function sub>, 4, 2, 1, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function add>]
555 [100, 75, 10, <built-in function sub>, 4, 2, <built-in function add>, 1, <built-in

## References
***

[[1]](https://british-game-show.fandom.com/wiki/Countdown) 
 British Game Show Wiki <br>
[[2]](https://www.cambridge.org/core/journals/journal-of-functional-programming/article/countdown-problem/FCF81E290B194D1ACBF699842D4D4CA5)  HUTTON, G. (2002). The countdown problem. Journal of Functional Programming, 12(6), 609-616. doi:10.1017/S0956796801004300 <br>
[[3]](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.86.1168&rep=rep1&type=pdf) R. Hinze, ‘Countdown problem’. 2002. <br>
[[4]](https://arxiv.org/pdf/1502.05450.pdf) J.-M. Alliot, ‘The (Final) countdown’. arXiv, 2015.<br>


***
# End